### Basic working of Google Palm LLM in LangChain

In [1]:
#!pip install --user google-generativeai torch

In [2]:
#!pip install -U sentence-transformers


In [3]:
#!pip install InstructorEmbedding

In [4]:
#!pip install --upgrade ipywidgets
#jupyter nbextension enable --py widgetsnbextension

In [2]:
import tqdm as notebook_tqdm
from langchain.llms import GooglePalm

api_key = 'GOOGLE_API_KEY' # get this free api key from https://makersuite.google.com/

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [3]:
poem = llm("Write a 4 line poem of my love for vadapaw")
print(poem)

Vada pav, oh vada pav,
You're the best snack in the land.
Your crispy shell, your soft filling,
Your tangy sauce, they make me sing.


In [7]:
essay = llm("write email to recruter for interview")
print(essay)

Dear [Hiring Manager name],

I am writing to express my interest in the [job title] position that I saw advertised on [website]. I have been working in the [industry] industry for [number] years, and I have a proven track record of success in [list of accomplishments].

In my previous role at [company name], I was responsible for [list of responsibilities]. I successfully managed a team of [number] employees and achieved [list of results]. I am confident that I have the skills and experience that you are looking for in a [job title].

I am eager to learn more about the [job title] position and how I can contribute to your team. I am available for an interview at your earliest convenience.

Thank you for your time and consideration.

Sincerely,
[Your name]


In [22]:
import json
import csv

# Specify the path to your JSON file
file_path = 'Ecommerce_FAQ_Chatbot_dataset.json'

# Load the JSON data
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Create a list to store question and answer pairs
questions_and_answers = []

# Access and process the questions and answers
for question_data in data['questions']:
    question = question_data['question']
    answer = question_data['answer']
    questions_and_answers.append((question, answer))

# Specify the CSV output file name
csv_file = 'Ecommerce_FAQ_Chatbot_dataset.csv'

# Write the data to a CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write the header row
    csv_writer.writerow(['Question', 'Answer'])
    
    # Write the question and answer pairs
    csv_writer.writerows(questions_and_answers)

print("Data has been saved as CSV in", csv_file)


Data has been saved as CSV in Ecommerce_FAQ_Chatbot_dataset.csv


In [23]:
data = pd.read_csv("Ecommerce_FAQ_Chatbot_dataset.csv")
data.head()

,Question,Answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


In [12]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

### Now let's load data from Amazon FAQ csv file

In [26]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='Ecommerce_FAQ_Chatbot_dataset.csv',source_column="Question")

# Store the loaded data in the 'data' variable , source_column="question"
data = loader.load()

In [28]:
data[1]

Document(page_content='Question: What payment methods do you accept?\nAnswer: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.', metadata={'source': 'What payment methods do you accept?', 'row': 1})

### Hugging Face Embeddings

In [30]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings()

e = instructor_embeddings.embed_query("What payment methods do you accept?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [23]:
len(e)

768

In [31]:
e[:5]

[-0.05164218693971634,
 -0.015128973871469498,
 -0.035818688571453094,
 -0.02933351881802082,
 0.038490913808345795]

As you can see above, embedding for a sentance "What is your refund policy" is a list of size 768. Looking at the numbers in this list, doesn't give any intuitive understanding of what it is but just assume that these numbers are capturing the meaning of "What is your refund policy". If you are curious to know about embeddings, go to youtube and search "codebasics word embeddings" and you will find bunch of videos with simple, intuitive explanations

### Vector store using FAISS

In [32]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [33]:
rdocs = retriever.get_relevant_documents("What payment methods do you accept?")
rdocs

[Document(page_content='Question: What payment methods do you accept?\nAnswer: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.', metadata={'source': 'What payment methods do you accept?', 'row': 1}),
 Document(page_content='Question: Are my personal and payment details secure?\nAnswer: Yes, we take the security of your personal and payment details seriously. We use industry-standard encryption and follow strict security protocols to ensure your information is protected.', metadata={'source': 'Are my personal and payment details secure?', 'row': 13}),
 Document(page_content='Question: Can I request an invoice for my order?\nAnswer: Yes, an invoice is usually included with your order. If you require a separate invoice, please contact our customer support team with your order details.', metadata={'source': 'Can I request an invoice for my order?', 'row': 37}),
 Document(page_content='Question: Can I order by phone?\nAnswer: Unfortunately, we do 

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

##### Embeddings can be created using GooglePalm too. Also for vector database you can use chromadb as well as shown below. During our experimentation, we found hugging face embeddings and FAISS to be more appropriate for our use case

In [14]:
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)

# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

### Create RetrievalQA chain along with prompt template 🚀

In [37]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            #return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


### We are all set 👍🏼 Let's ask some questions now

In [38]:
chain('What payment methods do you accept?')

{'query': 'What payment methods do you accept?',
 'result': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'}

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**

In [39]:
chain("Do you guys offer EMI payments?")

{'query': 'Do you guys offer EMI payments?',
 'result': 'Yes, we offer EMI payments on all orders above Rs. 5000.'}

In [40]:
chain("How can I track my order?")

{'query': 'How can I track my order?',
 'result': "\nAnswer: You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment."}

In [41]:
chain("What is your return policy?")

{'query': 'What is your return policy?',
 'result': 'Response: We offer a 30-day return policy on all products. If you are not satisfied with your purchase, you can return it for a full refund.'}

In [42]:
chain("tell me about return policy?")

{'query': 'tell me about return policy?',
 'result': 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.'}

In [43]:
chain("How long does shipping take?")

{'query': 'How long does shipping take?',
 'result': '\nAnswer: Shipping times vary depending on the destination and the shipping method chosen. Standard shipping usually takes 3-5 business days, while express shipping can take 1-2 business days.'}

In [45]:
chain("Can I plase order by my phone?")

{'query': 'Can I plase order by my phone?',
 'result': 'Unfortunately, we do not accept orders over the phone. Please place your order through our website for a smooth and secure transaction.'}

In [46]:
chain("How can I create an account?")

{'query': 'How can I create an account?',
 'result': "\nAnswer: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."}